In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_sku_adhoc
  TBLPROPERTIES (
    'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
    'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
  )
  AS
  SELECT * EXCEPT (proc_date, upload_user, upload_time)
  FROM {catalog_name}.udp_wcm_silver_dataportal.d_mdq_sku_adhoc
      WHERE upload_time = (SELECT MAX(upload_time) FROM {catalog_name}.udp_wcm_silver_dataportal.d_mdq_sku_adhoc)
      QUALIFY ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY upload_time DESC) = 1 ;
""")